# 📓 QuickRAG-FAQ-Bot Demo

### 🔹 QuickRAG-FAQ-Bot Architecture

``` mermaid
flowchart TD
    A[User Query] --> B[Embed with SentenceTransformers]
    B --> C[Qdrant Cloud Vector DB]
    C --> D[Retrieve Top-k Relevant Docs]
    D --> E[Groq LLM: LLaMA 3.1 8B]
    E --> F[Final Answer with Citations]
```

🔹 1. Install Requirements

In [8]:
%pip install -r ../requirements.txt
print("[LOG] Requirements installed.")


Note: you may need to restart the kernel to use updated packages.
[LOG] Requirements installed.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


🔹 2. Load Environment Keys

In [9]:
from dotenv import load_dotenv
import os, getpass

load_dotenv()
print("[LOG] .env loaded.")

# Prompt only if missing
for key in ["QDRANT_URL", "QDRANT_API_KEY", "GROQ_API_KEY"]:
    if not os.getenv(key):
        prompt_text = f"{key} not found in .env. Paste it now (won't be saved):"
        os.environ[key] = getpass.getpass(prompt_text)

# Ensure keys exist
if not os.getenv("QDRANT_URL") or not os.getenv("QDRANT_API_KEY"):
    raise RuntimeError("Qdrant keys are required to run this notebook.")
if not os.getenv("GROQ_API_KEY"):
    print("[WARN] Groq key missing. Only local fallback generation will work.")

print("[LOG] Keys status -> Qdrant:", bool(os.getenv("QDRANT_URL")), 
      "Groq:", bool(os.getenv("GROQ_API_KEY")))
print("[LOG] Environment ready.")

[LOG] .env loaded.
[LOG] Keys status -> Qdrant: True Groq: True
[LOG] Environment ready.


🔹 3. Index Data to Qdrant Cloud

In [10]:
import os, pandas as pd

csv_path = "../data/rag_optimized_5000.csv"

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV file not found at {csv_path}")

df = pd.read_csv(csv_path)
print(f"[LOG] Loaded {len(df)} rows from CSV.")

# Run indexing script
!python ../src/load_data.py --csv ../data/rag_optimized_5000.csv --max 2000
print("[LOG] Indexing complete. Docs stored in Qdrant Cloud.")


[LOG] Loaded 5000 rows from CSV.
[LOG] Indexing 2000 document chunks...
[QdrantDB] Collection 'quickrag_collection' ready with vector size 384
[RAG] Indexing 2000 docs with batch_size=64
[Embedder] Embedding 2000 texts with batch_size=64
[Embedder] Batch 1: 64 texts
[Embedder] Batch 2: 64 texts
[Embedder] Batch 3: 64 texts
[Embedder] Batch 4: 64 texts
[Embedder] Batch 5: 64 texts
[Embedder] Batch 6: 64 texts
[Embedder] Batch 7: 64 texts
[Embedder] Batch 8: 64 texts
[Embedder] Batch 9: 64 texts
[Embedder] Batch 10: 64 texts
[Embedder] Batch 11: 64 texts
[Embedder] Batch 12: 64 texts
[Embedder] Batch 13: 64 texts
[Embedder] Batch 14: 64 texts
[Embedder] Batch 15: 64 texts
[Embedder] Batch 16: 64 texts
[Embedder] Batch 17: 64 texts
[Embedder] Batch 18: 64 texts
[Embedder] Batch 19: 64 texts
[Embedder] Batch 20: 64 texts
[Embedder] Batch 21: 64 texts
[Embedder] Batch 22: 64 texts
[Embedder] Batch 23: 64 texts
[Embedder] Batch 24: 64 texts
[Embedder] Batch 25: 64 texts
[Embedder] Batch 26: 


Embedding batches: 100%|██████████| 32/32 [00:17<00:00,  1.81it/s]
Traceback (most recent call last):
  File "d:\Naman\Projects\RAG\QuickRAG-FAQ-Bot\venv\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "d:\Naman\Projects\RAG\QuickRAG-FAQ-Bot\venv\Lib\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Naman\Projects\RAG\QuickRAG-FAQ-Bot\venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "d:\Naman\Projects\RAG\QuickRAG-FAQ-Bot\venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Naman\Projects\RAG\QuickRAG-FAQ-Bot\venv\Lib\site-packages\httpcore\_sync\connection.py", line 103, in handle_request
    return self._connection.han

🔹 4. Setup RAG

In [11]:
import sys, os
sys.path.append(os.path.abspath("../src"))
print("[LOG] src/ added to sys.path")

from rag import RAG

# Initialize RAG (Qdrant + Groq)
rag = RAG()
print("[LOG] RAG pipeline ready.")


[LOG] src/ added to sys.path
[QdrantDB] Collection 'quickrag_collection' ready with vector size 384
[LOG] RAG pipeline ready.


🔹 5. Single Query Demo

In [12]:
query = "How do I reset my password?"
print("[LOG] Query:", query)

# Retrieve relevant docs
retrieved = rag.retrieve(query, top_k=5)
print(f"[LOG] Retrieved {len(retrieved)} docs:")
for i, doc in enumerate(retrieved, 1):
    print(f"Doc {i}:", doc[:120], "...")

# Generate answer
print("\n[LOG] Generating answer...")
answer = rag.answer(query, top_k=5)
print("\n[Answer]:\n", answer)


[LOG] Query: How do I reset my password?
[RAG] Retrieving top 5 docs for query: How do I reset my password?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 45.46it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[LOG] Retrieved 0 docs:

[LOG] Generating answer...
[RAG] Answering query: How do I reset my password?
[RAG] Retrieving top 5 docs for query: How do I reset my password?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 90.92it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[RAG] Using Groq for generation.

[Answer]:
 To reset your password, follow these steps:

1. Go to the login page of the account you want to reset the password for.
2. Click on the "Forgot Password" or "Reset Password" link.
3. Enter the email address or username associated with your account.
4. Follow the instructions sent to your email to reset your password.
5. Create a new password and confirm it.

Retrieved Documents: 
- User Guide to Account Recovery (Help Center, Section 4.2)
- Password Reset Policy (Terms of Service, Article 3.4)
- Account Management FAQ (Knowledge Base, Question 12)


🔹 6. Multi-Query Demo

In [13]:
queries = [
    "When is the library open?",
    "How do I reset my password?"
]

for query in queries:
    print("\n==============================")
    print("[LOG] Query:", query)
    
    # Retrieve top docs
    retrieved = rag.retrieve(query, top_k=5)
    print(f"[LOG] Retrieved {len(retrieved)} docs:")
    for i, doc in enumerate(retrieved, 1):
        print(f"Doc {i}:", doc[:120], "...")
    
    # Generate answer
    print("\n[LOG] Generating answer...")
    answer = rag.answer(query, top_k=5)
    print("\n[Answer]:\n", answer)



[LOG] Query: When is the library open?
[RAG] Retrieving top 5 docs for query: When is the library open?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 19.61it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[LOG] Retrieved 0 docs:

[LOG] Generating answer...
[RAG] Answering query: When is the library open?
[RAG] Retrieving top 5 docs for query: When is the library open?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 77.01it/s]

...

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[LOG] Retrieved 0 docs:

[LOG] Generating answer...
[RAG] Answering query: How do I reset my password?
[RAG] Retrieving top 5 docs for query: How do I reset my password?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 100.10it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[RAG] Using Groq for generation.

[Answer]:
 To reset your password, follow these steps:

1. Go to the sign-in page of the application or website.
2. Click on the "Forgot password" or "Reset Password" link.
3. Enter your email address or username associated with your account.
4. Follow the instructions sent to your email to reset your password.

Retrieved documents:

- General password reset procedures for most web applications [1]
- Password reset protocol for specific applications (e.g., Google, Facebook) [2]

Please note: The steps might vary slightly depending on the specific application or website.

References:
[1] - General password reset procedures for most web applications.
[2] - Password reset protocol for specific applications (e.g., Google, Facebook).
